# <span style="color:maroon">**Cleaning Data in Python**</span>

## <span style="color:blue">**Functions and Methods**</span>

##### pandas functions
> pandas.read_csv(file, header, names, index_col, parse_dates)  
> pandas.melt(df, id_vars, value_vars, var_names, value_names)  
> pandas.pivot_table(df, index columns, values, aggfunc)  
> pandas.concat(list, ignore_index) **--> list is a list of dataframes**  
> pandas.merge(left, right, left_on, right_on)  
> pandas.to_numeric(df.column_name, errors)  **--> errors="coerce" coerces type conversion errors to NaN**

##### glob functions
> glob.glob(pattern)  

##### re functions
> re.compile(regex)  **--> instantiates an re.Pattern**  
> re.findall(regex, string) **--> useful when looking for numbers in text; in that case regex would be something like "\d+"**  
> re.match(regex, string)  

##### re methods
> re_Pattern_object.match(string)  **--> if string matches the regex then an re.Match object is returned else None is return**  

##### dataframe methods
> df.head(n)  
> df.tail(n)  
> df.info()  
> df.describe()  
> df["column_name"].value_counts(dropna) **--> equivalent to -->** df.column_name.value_counts(dropna)  

##### dataframe attributes
> df.shape  
> df.columns  
> df.dtypes  
> df.index

##### dataframe manipulation
> df["column_to_be_added"] = df["pre-existing_column"].str[0] **--> str is an example**  
> df["split_column_name"] = df["column_to_be_split"].str.split("\_") **--> split_column_name is a list; "\_" is an example**  
> df["column_to_be_added] = df["split_column_name"].str.get(n) **-> n is the index value for the list in "split_column_name"**  
> df["column_name"] = df["column_name"].astype("category")

##### list manipulation
> list.append(object) **--> appends an object to the tail end of a list**

## <span style="color:blue">**Exploring Your Data**</span>

##### Common Data Problems
- Inconsistent column names
- Missing data
- Outliers
- Duplicate rows
- Untidy data structure
- Need to process columns
- Column types can signal unexpected data values

In [ ]:
import pandas as pd

file = "./data/stock_data.csv"
column_names = ["ticker", "date", "close", "cap", "volume"]
df = pd.read_csv(file, header=1, names=column_names, index_col="date", parse_dates=True)

In [ ]:
print(df.head(), "\n")
print(df.tail(), "\n")
print(df.shape, "\n")
print(df.columns, "\n")
print(df.info(), "\n")
print(df.describe(), "\n")
print(df.ticker.value_counts(dropna=False), "\n")
print(df["ticker"].value_counts(dropna=False), "\n")     # same as previous lines


## <span style="color:blue">**Tidying Data for Analysis**</span>

##### Tidy Data
- "Tidy Data" - Hadley Wickham, Ph.D.
- Formalize the way we describe the shape of data
- Gives us a goal when formatting our data
- Standard way to organize data values within a dataset
- Certain data formats are better for reporting whereas other data formats are better for analysis
- Easier to fix common data problems
- Columns containing values, instead of variables

##### Tidy Data Tenets
- Columns represent separate variables
- Rows represent individual observations
- Observation units form tables

In [ ]:
# Tidy and Untidy Data
import pandas as pd
file = "./data/airquality.csv"
airquality = pd.read_csv(file)
print(airquality.head(), "\n")                 # airquality is a tidy dataframe

# convert the tidy dataframe airquality into an untidy dataframe using pd.melt()
airquality_melt = pd.melt(airquality, id_vars=["Month", "Day"], value_vars=["Ozone", "Solar.R", "Wind", "Temp"])
print(airquality_melt.head())

In [ ]:
airquality_melt2 = pd.melt(airquality, id_vars=["Month", "Day"], var_name="measurement", value_name="reading")
print(airquality_melt2.head())

##### Pivoting Data
- Inverse of melting data
- Pivoting takes an analysis-friendly shape (untidy) and creates a reporting-friendly shape (tidy)
- Used to take a dataset that violates tidy data principle and converts to a tidy dataset

In [ ]:
import numpy as np
airquality_unmelted = pd.pivot_table(airquality_melt2, index=["Month", "Day"],
                                     columns="measurement", values="reading", aggfunc=np.mean)
print(airquality_unmelted.head())
# notice what happens to the "Month" index --> this is a hierarchical index - also known as a multi-index

In [ ]:
print(airquality_unmelted.index, "\n")
airquality_reborn = airquality_unmelted.reset_index()
print(airquality_reborn.index, "\n")
print(airquality_reborn.head(), "\n")

In [ ]:
tb = pd.read_csv("./data/tb.csv")
print(tb.head(), "\n")

tb_melt = pd.melt(tb, id_vars=["country", "year"])
tb_melt["gender"] = tb_melt.variable.str[0]            # create a new column that is a calculation based on another column
tb_melt["age_group"] = tb_melt.variable.str[1:]        # create a new column that is a calculation based on another column

print(tb_melt.head())

In [ ]:
ebola = pd.read_csv("./data/ebola.csv")
print(ebola.head(), "\n")

ebola_melt = pd.melt(ebola, id_vars=["Date", "Day"], var_name="type_country", value_name="counts")
print(ebola_melt.head(), "\n")

ebola_melt["str_split"] = ebola_melt["type_country"].str.split("_")      # returns a list
print(ebola_melt.head(), "\n")

ebola_melt["type"] = ebola_melt["str_split"].str.get(0)

ebola_melt["country"] = ebola_melt["str_split"].str.get(1)

print(ebola_melt.head())

## <span style="color:blue">**Combining Data for Analysis**</span>

In [ ]:
import pandas as pd

file1 = "./data/stock_data_1.csv"
file2 = "./data/stock_data_2.csv"
column_names = ["ticker", "date", "close", "cap", "volume"]
df1 = pd.read_csv(file1, header=1, names=column_names, index_col="date", parse_dates=True)
df2 = pd.read_csv(file2, header=1, names=column_names, index_col="date", parse_dates=True)

print(df1.shape, "\n")
print(df2.shape, "\n")

In [ ]:
df = pd.concat([df1, df2], ignore_index=True)      # pd.concat requires a list
                                                   # axis=0 concatenates row-wise (default), axis=1 concatenates column-wise
print(df.shape)

In [ ]:
import pandas as pd
import glob

def combine_dataframes(pattern, names, index_col, na_values, header=1, parse_dates=True, ignore_index=True):
    # creates a list of file pointers matching the pattern
    files = glob.glob(pattern)                        
    frames = []
   
    # iterate through each file pointer
    for f in files:
    
        # loads file into dataframe variable: df
        df = pd.read_csv(f, names=names, index_col=index_col, na_values=na_values)                           
        
        # adds dataframe from df into list: frames
        frames.append(df)  
    
    # concatenate all dataframes in frames into a single dataframe
    return pd.concat(frames, ignore_index=ignore_index)

#### merge left and right dataframes using a 1-to-1 join
> o2o = pd.merge(left=site, right=visited, left_on="name", right_on="site")

This is actually a full outer join. Note what happens when one of the dataframes does NOT match on the join.\
This behavior is the same for 1-to-1, many-to-1, and many-to-many joins

## <span style="color:blue">**Cleaning Data for Analysis**</span>

In [ ]:
import pandas as pd

print(tips.dtypes, "\n")                               # gets the datatype of each column in a dataframe
tips = pd.read_csv("./data/tips.csv")
print(tips.info(), "\n")
tips["sex"] = tips["sex"].astype("category")           # note that this is the same --> tips.sex.astype("category") = tips.sex.astype("category")
tips["smoker"] = tips["smoker"].astype("category")
print(tips.info())

In [ ]:
tips.total_bill = pd.to_numeric(tips.total_bill, errors="coerce")
tips["total_bill"] = pd.to_numeric(tips["total_bill"], errors="coerce")

In [ ]:
# Import the regular expression module
import re

# Compile the pattern: prog
prog = re.compile("\d{3}-\d{3}-\d{4}")

# See if the pattern matches
result1 = prog.match("123-456-7890")
print(bool(result1))

# See if the pattern matches
result2 = prog.match("1123-456-7890")
print(bool(result2))

# Find the numeric values: matches
matches = re.findall("\d+", "the recipe calls for 10 strawberries and 1 banana")
print(matches)

pattern1 = bool(re.match(pattern=re.compile("^\d{3}-\d{3}-\d{4}$"), string="123-456-8330"))
print(pattern1)

pattern2 = bool(re.match(pattern="^\$\d*\.\d{2}$", string="$123.35"))
print(pattern2)

pattern3 = bool(re.match(pattern="^[A-Z]\w*$", string='Australia'))
print(pattern3)

In [ ]:
# Import the regular expression module
import re

# Compile the pattern: prog
prog = re.compile("\d{3}-\d{3}-\d{4}")

# See if the pattern matches
result1 = prog.match("123-456-7890")
print(bool(result1))

# See if the pattern matches
result2 = prog.match("1123-456-7890")
print(bool(result2))

# Find the numeric values: matches
matches = re.findall("\d+", "the recipe calls for 10 strawberries and 1 banana")
print(matches)

pattern1 = bool(re.match(pattern=re.compile("^\d{3}-\d{3}-\d{4}$"), string="123-456-8330"))
print(pattern1)

pattern2 = bool(re.match(pattern="^\$\d*\.\d{2}$", string="$123.35"))
print(pattern2)

pattern3 = bool(re.match(pattern="^[A-Z]\w*$", string='Australia'))
print(pattern3)

## <span style="color:blue">**Case Study**</span>

## <span style="color:blue">**Miscellaneous**</span>